In [3]:
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import numpy as np
import time

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Chrome/125.0.6422.76"}) # Safari/537.36. Chrome/103.0.0.0
    soup = bts(result.text, "html.parser")
    return soup

# Açıklamalar

In [5]:
#Aydın iline ait sıcaklık tahminleri elde edildi.
#Siteyi ilk açtığımızdaki bugüne ait değerler 
#https://www.accuweather.com/tr/tr/aydin/317040/weather-today/317040 linkiyle geliyor. 
#İkinci güne ait link https://www.accuweather.com/tr/tr/aydin/317040/weather-tomorrow/317040 şeklinde geliyor. 
#Üçüncü günden itibaren https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=3 şeklinde geliyor.
#Linkler değişiyor. Ancak eğer biz day=1 yaparsak bugüne, day=2 yaparsak yarının değerlerine ulaşabiliyoruz.
#Bu yüzden döngü ile rahatlıkla linkleri çekebildik.
#Her sayfada gece ve gündüze ait değerleri elde etmemiz isteniyor.
#Bu değerler aynı iki parametrede bulunuyor. (half-day-card-header__title)
#Bu yüzden 2 adet index içeren bir döngüyle gündüz ve gece verilerini rahatlıkla çektik.
#for index in range (0,2): işleminin sebebi gündüz ve gece verilerine ayrı ayrı ulaşabilmektir.
#Gece verilerinde Maks UV İndeksi olmadığı için NAN gelmiştir. Bu durum olağandır.

In [6]:
pages = ["https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=1"]
for page in range(2,8):
    pages.append("https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day="+str(page))
    
pages

['https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=1',
 'https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=2',
 'https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=3',
 'https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=4',
 'https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=5',
 'https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=6',
 'https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=7']

In [7]:
html=getAndParseURL('https://www.accuweather.com/tr/tr/aydin/317040/daily-weather-forecast/317040?day=1')

In [8]:
date_all=html.find("div", {"class": "subnav-pagination"}).text.strip().split()
day=date_all[0].rstrip(',')
day

'Pazar'

In [9]:
date=date_all[1] + ' ' + date_all[2]
date

'23 Haziran'

In [10]:
day_period=html.find("div", {"class": "half-day-card-header__title"}).text.strip().split()[0]
day_period

'Gündüz'

In [11]:
temperature=html.find("div", {"class":"temperature"}).text.strip()
temperature

'39°En Yüks.'

In [12]:
max_uv_index=html.find("div", {"class":"left"}).find(string="Maks UV İndeksi").findNext().text.strip()
max_uv_index

'11 Çok Sağlıksız'

In [13]:
possibility_of_thundershowers=html.find("div",{"class": "right"}).find(string="Gök Gürültülü Sağanak Yağış Olasılığı").findNext().text.strip()
possibility_of_thundershowers

'%0'

In [14]:
wind=html.find("div", {"class":"left"}).find(string="Rüzgar").findNext().text.strip()
wind

'BGB 6 km/sa'

In [15]:
rainfall=html.find("div", {"class":"right"}).find(string="Yağış").findNext().text.strip()
rainfall

'0.0 mm'

In [16]:
rainfall_forecast=html.find("div", {"class":"left"}).find(string="Yağış Beklentisi").findNext().text.strip()
rainfall_forecast

'%0'

In [17]:
day_other_period=html.find("div", {"class":"half-day-card-header__title"}).text.strip()
day_other_period

'Gündüz\n\n\t\t\t\t23.6'

In [18]:
datas=html.find_all("div", {"class":"half-day-card-header__title"})[1].text.strip().split()[0]
datas

'Gece'

In [36]:
df_features=[]
for page in pages:
    html=getAndParseURL(page)
    date_all=html.find("div", {"class": "subnav-pagination"}).text.strip().split()
    for index in range (0,2):
        try:
            day=day=date_all[0].rstrip(',')
        except:
            day=np.nan
        try:
            date=date_all[1] + ' ' + date_all[2]
        except:
            date=np.nan
        try:
            day_period=html.find_all("div", {"class":"half-day-card-header__title"})[index].text.strip().split()[0]
        except:
            day_period=np.nan
        try:
            temperature=html.find_all("div", {"class":"temperature"})[index].text.strip()
        except:
            temperature=np.nan
        try:
            wind=html.find_all("div", {"class":"left"})[index].find(string="Rüzgar").findNext().text.strip()
        except:
            wind=np.nan
        try: 
            possibility_of_thundershowers=html.find_all("div",{"class": "right"})[index].find(string="Gök Gürültülü Sağanak Yağış Olasılığı").findNext().text.strip()
        except:
            possibility_of_thundershowers=np.nan
        try:
            rainfall=html.find_all("div", {"class":"right"})[index].find(string="Yağış").findNext().text.strip()
        except:
            rainfall=np.nan
        try:
            rainfall_forecast=html.find_all("div", {"class":"left"})[index].find(string="Yağış Beklentisi").findNext().text.strip()
        except:
            rainfall_forecast=np.nan
        try:
            covered_in_clouds=html.find_all("div", {"class":"right"})[index].find(string="Bulutlarla Kaplı").findNext().text.strip()
        except:
            covered_in_clouds=np.nan
        try:
            max_uv_index=html.find_all("div", {"class":"left"})[index].find(string="Maks UV İndeksi").findNext().text.strip()
        except:
            max_uv_index=np.nan

        df_features.append([date, day, day_period, temperature, wind, rainfall_forecast, possibility_of_thundershowers, rainfall, covered_in_clouds, max_uv_index])
        
        time.sleep(1)

    
df_columns=["Tarih", "Gün", "Gün Dönemi", ";Sıcaklık", "Rüzgar", "Yağış Beklentisi", "Gök Gürültülü Sağanak Yağış Olasılığı", "Yağış", "Bulutlarla Kaplı", "Maks UV İndeksi"]

df = pd.DataFrame.from_records(df_features, columns=df_columns)
            

In [37]:
df

,Tarih,Gün,Gün Dönemi,;Sıcaklık,Rüzgar,Yağış Beklentisi,Gök Gürültülü Sağanak Yağış Olasılığı,Yağış,Bulutlarla Kaplı,Maks UV İndeksi
0,23 Haziran,Pazar,Gündüz,39°En Yüks.,BGB 6 km/sa,%0,%0,0.0 mm,%0,11 Çok Sağlıksız
1,23 Haziran,Pazar,Gece,21°En Düş.,KB 6 km/sa,%0,%0,0.0 mm,%0,NaN
2,24 Haziran,Pazartesi,Gündüz,40°En Yüks.,BGB 13 km/sa,%0,%0,0.0 mm,%2,11 Çok Sağlıksız
3,24 Haziran,Pazartesi,Gece,21°En Düş.,BKB 7 km/sa,%0,%0,0.0 mm,%0,NaN
4,25 Haziran,Salı,Gündüz,40°En Yüks.,B 13 km/sa,%0,%0,0.0 mm,%0,11 Çok Sağlıksız
5,25 Haziran,Salı,Gece,21°En Düş.,KKB 7 km/sa,%0,%0,0.0 mm,%0,NaN
6,26 Haziran,Çarşamba,Gündüz,41°En Yüks.,BKB 11 km/sa,%0,%0,0.0 mm,%0,11 Çok Sağlıksız
7,26 Haziran,Çarşamba,Gece,21°En Düş.,KB 6 km/sa,%0,%0,0.0 mm,%0,NaN
8,27 Haziran,Perşembe,Gündüz,41°En Yüks.,BGB 11 km/sa,%0,%0,0.0 mm,%0,11 Çok Sağlıksız
9,27 Haziran,Perşembe,Gece,22°En Düş.,BKB 6 km/sa,%0,%0,0.0 mm,%0,NaN


In [39]:
df.to_csv('/Users/nyzmemre/dsBootCamp/datas/acuweather_memre_odev.csv', index=False)